In [1]:
import artm
import jsonlines
import operator
import codecs
from collections import Counter
from tqdm import tqdm

import os
os.environ['ARTM_SHARED_LIBRARY']='C:/BigARTM/bin/artm.dll'

# Перобразовать в wabbit формат, если ещё не

In [ ]:
texts = [(item['text'], item['id']) for item in jsonlines.open("sources/normalized_texts.jl", 'r')]
texts.sort(key=operator.itemgetter(1))

In [ ]:
with codecs.open("wabbit", "w", encoding='utf-8') as output:
    for text in tqdm(texts):
        cnt = Counter(text[0].split())
        output.write(text[1] + ' ' + ' '.join([item + ':' + str(cnt[item]) for item in cnt]) + '\n')

# Применяем ARTM

In [2]:
num_topics = len([line for line in codecs.open("sources/accepted_categories.txt", "r", encoding='utf-8')])
num_topics

14697

In [3]:
batch_vectorizer = artm.BatchVectorizer(data_path='wabbit',
                                        data_format='vowpal_wabbit',
                                        target_folder='my_collection_batches')

'''batch_vectorizer = artm.BatchVectorizer(data_format='bow_n_wd',
                                        n_wd=n_wd,
                                        vocabulary=vocabulary,
                                        target_folder='my_collection_batches')'''

"batch_vectorizer = artm.BatchVectorizer(data_format='bow_n_wd',\n                                        n_wd=n_wd,\n                                        vocabulary=vocabulary,\n                                        target_folder='my_collection_batches')"

In [ ]:
dictionary = artm.Dictionary()
dictionary.gather(data_path='my_collection_batches')

In [ ]:
model = artm.ARTM(num_topics=num_topics, dictionary=dictionary,
                  show_progress_bars=True,
                  regularizers=[artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_regularizer')])

In [ ]:
model.scores.add(artm.PerplexityScore(name='my_fisrt_perplexity_score',
                                      dictionary=dictionary))

In [ ]:
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=10)

In [15]:
print(model.score_tracker['my_fisrt_perplexity_score'].value)

[16.69302749633789, 10.504449844360352, 10.16703987121582, 10.024514198303223, 9.9529447555542, 9.906949996948242, 9.850658416748047, 9.806109428405762, 9.779677391052246, 9.762627601623535]


In [24]:
model.get_phi_dense()

(array([[1.38047524e-02, 1.68282873e-04],
        [2.10436134e-04, 1.44476116e-01],
        [3.01596433e-01, 1.86692656e-03],
        [1.02518834e-01, 8.19944319e-07],
        [8.82437676e-02, 7.78485229e-03],
        [2.56709231e-04, 1.44798368e-01],
        [7.41048716e-04, 7.43682906e-02],
        [2.20018649e-07, 3.40052545e-01],
        [8.29713270e-02, 2.54379091e-04],
        [1.68291315e-01, 3.20942164e-03],
        [2.67181949e-05, 3.39664705e-02],
        [9.69519260e-06, 1.63328439e-01],
        [1.24874704e-01, 2.84956805e-05],
        [1.08825387e-02, 1.29033893e-07],
        [3.62287238e-02, 8.64593370e-04],
        [3.29797752e-02, 5.61207905e-02],
        [1.57055985e-02, 7.27119058e-08],
        [5.22775099e-06, 2.87109166e-02],
        [1.53344609e-02, 2.07113402e-07],
        [5.31766843e-03, 4.60314133e-13]], dtype=float32),
 ['al',
  'antoninus',
  'aim',
  'adium',
  'allowed',
  'allows',
  'and',
  'anonymous',
  'android',
  'ah',
  'apache',
  'altavista',
  '

In [23]:
sum([t[0] for t in p[0]])

0.9999999539841724

In [9]:
model.initialize(dictionary=dictionary)

NameError: name 'model' is not defined